In [6]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bokeh
import bokeh.palettes
from bokeh.plotting import figure, output_notebook, show, output_file, gridplot
from bokeh.io import export_svg, export_png
from bokeh.models import Legend, LegendItem
import os
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_adjacencies


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:v1.2.1', token=auth_token)
except:
    np_client = None

output_notebook()

Loading BokehJS ...

In [5]:
all_neuron_data, all_roi = fetch_neurons(NC(status='Traced'))
all_neuron_data

,bodyId,instance,type,pre,post,downstream,upstream,mito,size,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
0,198469830,None,None,0,0,0,0,2,6665418,Traced,True,Leaves,None,299.0,"[33045, 15655, 3280]",{},None,[],[]
1,198810827,None,None,0,0,0,0,1,9211633,Traced,True,Leaves,None,299.0,"[33014, 16221, 3088]",{},None,[],[]
2,198815212,None,None,0,0,0,0,3,27720313,Traced,True,Leaves,None,299.0,"[33342, 16128, 3216]",{},None,[],[]
3,199147536,None,None,0,0,0,0,3,58580023,Traced,True,Leaves,None,299.0,"[32412, 16453, 3356]",{},None,[],[]
4,199156210,None,None,0,0,0,0,4,82402138,Traced,True,Leaves,None,299.0,"[33401, 16741, 3240]",{},None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97895,7112622044,LAL137(PVL05)_L,LAL137,1376,2087,11635,2087,534,1529252938,Traced,False,Roughly traced,None,NaN,None,"{'SNP(R)': {'pre': 304, 'post': 130, 'downstre...",VeLC-(LC)(c)Sm(b),"[CRE(-ROB,-RUB)(R), CRE(-RUB)(L), CRE(L), CRE(...","[CRE(-ROB,-RUB)(R), CRE(-RUB)(L), CRE(L), CRE(..."
97896,7112622236,None,None,5,79,22,79,23,37146103,Traced,False,Prelim Roughly traced,None,NaN,None,"{'SNP(R)': {'pre': 5, 'post': 79, 'downstream'...",None,"[SLP(R), SMP(R), SNP(R)]","[SLP(R), SMP(R), SNP(R)]"
97897,7112623021,None,None,104,503,445,503,74,97081386,Traced,True,Leaves,None,NaN,None,"{'AL(L)': {'pre': 104, 'post': 503, 'downstrea...",None,"[AL(L), AL-D(L), AL-DA2(L), AL-DA3(L), AL-DA4m...","[AL(L), AL-D(L), AL-DA2(L), AL-DA3(L), AL-DA4m..."
97898,7112623023,None,None,28,17,205,17,17,26237427,Traced,True,Leaves,None,NaN,None,"{'AL(L)': {'pre': 28, 'post': 17, 'downstream'...",None,"[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]","[AL(L), AL-D(L), AL-DC1(L), AL-DC4(L), AL-DL5(L)]"


In [7]:
# roi data
roi_all = fetch_adjacencies(all_neuron_data['bodyId'], all_neuron_data['bodyId'])
roi_all

  0%|          | 0/490 [00:00<?, ?it/s]

(           bodyId    type         instance
 0       200326126   CL141          CL141_L
 1       202916528    FS4C  FS4C(FQ12)_C4_R
 2       203248725    None             None
 3       203248801    None             None
 4       203248973    None             None
 ...           ...     ...              ...
 93624  7112622044  LAL137  LAL137(PVL05)_L
 93625  7112622236    None             None
 93626  7112623021    None             None
 93627  7112623023    None             None
 93628  7112623383    None             None
 
 [93629 rows x 3 columns],
          bodyId_pre  bodyId_post     roi  weight
 0         200326126    264083994  SLP(R)       3
 1         200326126    295816140  ICL(R)       2
 2         200326126    295816140  SCL(R)       2
 3         200326126    295816140  PLP(R)       1
 4         200326126    296203440  SCL(R)       1
 ...             ...          ...     ...     ...
 7375765  7112623383    823913632  SMP(L)       1
 7375766  7112623383    832802029  SMP(L)  